In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# **1. Install required packages:**

In [3]:
!pip install --q llama-index llama-index-llms-huggingface llama-index-llms-huggingface-api
!pip install --q llama-index-embeddings-huggingface vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

# **2. Import necessary classes:**

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM


# **4. Initialize the LLM using TinyLlama/TinyLlama-1.1B-Chat-v1.0 model:**

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [6]:
llm = HuggingFaceLLM(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    context_window=2048,
    max_new_tokens=256,
    device_map="auto"
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# **5. Set up the embedding model:**

In [7]:
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **6. Apply models to global settings:**

In [8]:
Settings.llm = llm
Settings.embed_model = embed_model

# **7. Create the index from documents:**

In [10]:
document = SimpleDirectoryReader("/content/paper").load_data()

In [11]:
index = VectorStoreIndex.from_documents(document)

# **8. Persist the index to disk:**

In [12]:
index.storage_context.persist(persist_dir="/content/index")

# **9. Query the index with natural language prompts:**

In [14]:
question = "Write a detailed summary of prompting techniques"
query = index.as_query_engine()
response = query.query(question)
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (2127 > 2048). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.




Prompting techniques are a crucial aspect of language learning, as they help learners to understand and remember new words and concepts. Here are some prompting techniques that can be used to improve your language learning:

1. Asking for feedback: Asking for feedback on your progress can help you understand and remember relevant material. For example, if you're learning a new word, ask yourself what it means and how it's used in context.

2. Interview: Asking questions that require you to think critically and analyze information can help you understand new concepts. For example, if you're learning a new language, ask yourself questions about grammar, vocabulary, and cultural differences.

3. Interpreting: Asking questions that require you to interpret information can help you understand new concepts. For example, if you're learning a new sport, ask yourself questions about technique, strategy, and injury prevention.

4. Understanding: Asking questions that require you to understand 

In [15]:
question = "What is fine-tuning of language models?"
query = index.as_query_engine()
response = query.query(question)
print(response)


Fine-tuning of language models is a technique for improving the performance of pre-trained language models for specific tasks. Fine-tuning a pre-trained language modeling:arsenal inch225,2022.
7. Theem
7.
Finish
72016
7:60
70
7/24/2020
76220/2020
78020
7/2020
7820/24
776
(76
For language
[2020/20
7/24/2/777/202/7/202/7
76/77/26/202/220/22
7/202
7/7/202/2020202
7/7/2222222
722/77/777/7/22222222220772222/22222


In [16]:
question = "Summarize the sparks of AGI paper"
query = index.as_query_engine()
response = query.query(question)
print(response)


The paper "Spark:00000000S2022F10-000000000000000000000000000000000000000000000000000000000
Spark" is about a new research paper that explores the use of deep learning techniques to improve the performance of machine translation systems. The paper is written by researchers from the University of California, Berkeley, and the University of Toronto, and it was published in the journal "arXiv preprint arXiv:2302.05442" in February 2023. The paper's main contributions include the development of a new language modeling approach called "Audiolm" and the use of multitask learning to improve audio generation. The paper also discusses the potential of deep learning for audio processing and the need for more research in this area. The paper's authors include researchers from the University of California, Berkeley, and the University of Toronto, as well as researchers from other
